In [ ]:
# import required libraries
import pandas as pd
import numpy as np
import geog
import simplekml
import shapely

In [ ]:
# read in dataframe of network data (this is a dummy dataset)
df = pd.read_csv('network_data.csv')
df.info()
# the dataframe contains: 
    # a cell identifier, 
    # cell locations, 
    # the number of connections to a cell, 
    # the average distance from the cell which users connect to 
    # and the direction the cell faces 


In [ ]:
# define functions to create different components required to create wedges

# starting with a circle...
def createCircle(lat,lng,n_points,distance_radius_meters):
    p = (lng, lat)
    n_points = n_points
    d = distance_radius_meters  # meters
    angles = np.linspace(0, 360, n_points)
    polygon = geog.propagate(p, angles, d)
    return shapely.geometry.Polygon(polygon)

# ...then two lines to intersect the circle and split it into a wedge and the remainder...
def createLine(lat,lng,n_points,distance_radius_meters,angle):
    p = (lng, lat)
    n_points = n_points
    d = distance_radius_meters  # meters
    polygon = geog.propagate(p, angle, d, bearing=True)
    try:
        return shapely.geometry.LineString([(lng, lat), (polygon[0], polygon[1])])
    except:
        return None
    
# ...and implement the intersection and pick the wedge by selecting the shape with the minimum area between the two 
#    (the wedge and the remainder)
def createWedge(circle, l1, l2):
    if (circle.is_valid) & (l1.is_valid) & (l2.is_valid):
        plist = circle.difference(l1.buffer(1e-6)).difference(l2.buffer(1e-6)).geoms
        return plist[[pol.area for pol in plist].index(min([pol.area for pol in plist]))]
    else:
        return np.nan

In [ ]:
# create circles for each location with radius (in meters) equal to the value in the 'distance from cell' column
# we vectorize the function to speed up the process
df['circle_geometry'] = np.vectorize(createCircle)(df['Latitude'], df['Longitude'], 20, df['Timing Advance'])

# here we specify the width of the wedge, with the angle applied to either side of the direction the cell is facing
# resulting in a wedge with 2 x the size of the cone span angle value
cone_span = 60

# create lines to cut the circle in the direction we want
df['l1'] = np.vectorize(createLine)(df['Latitude'], df['Longitude'], 2, df['Timing Advance'], df['Azimuth']-cone_span)
df['l2'] = np.vectorize(createLine)(df['Latitude'], df['Longitude'], 2, df['Timing Advance'], df['Azimuth']+cone_span)

# extract the wedge piece of the geometries
df['wedge_geometry'] = np.vectorize(createWedge)(df['circle_geometry'], df['l1'], df['l2'])

In [ ]:
# look at the distribution of user counts
df['LTE Users'].describe()

In [ ]:
# create a function to assign colours to each wedge based on the number of users
# the first two characters represent transparency which can be set from 00 to ff (which is invisible to opaque)
# you can use 6 digit hex code in reverse for the last six characters
def createColor(user_stats):
    if (user_stats>=df['LTE Users'].quantile(0.60)):
        return '501400FF'
    elif (user_stats>=df['LTE Users'].quantile(0.30)):
        return '5014B4FF'
    elif (user_stats>=1):
        return '5000FF14'          
    else:
        return '50F0FF14'

# apply the colour function
df['polycolor'] = np.vectorize(createColor)(df['LTE Users'])

# remove spaces in columns names as preparation for writing to kml
df.columns = [x.replace(' ','_') for x in df.columns]

In [ ]:
# create a function to assign colours to each wedge based on the number of users
# the first two characters represent transparency which can be set from 00 to ff (which is invisible to opaque)
# you can use 6 digit hex code in reverse for the last six characters
def createColor(user_stats):
    if (user_stats>=df['LTE Users'].quantile(0.60)):
        return '501400FF'
    elif (user_stats>=df['LTE Users'].quantile(0.30)):
        return '5014B4FF'
    elif (user_stats>=1):
        return '5000FF14'          
    else:
        return '50F0FF14'

# apply the colour function
df['polycolor'] = np.vectorize(createColor)(df['LTE Users'])

# remove spaces in columns names as preparation for writing to kml
df.columns = [x.replace(' ','_') for x in df.columns]

# create a kml with all of the cells
# the name of each polygon will be the cell id, so that its easy to search for a particular cell after importing into GoogleEarth
# the description is a html pop up object that displays som supplementary information
kml = simplekml.Kml()
for row in df.sort_values(by=['LTE_Users'], ascending=False).itertuples():
    pol = kml.newpolygon(name=f'{row.Cellname}', 
                         description=f"""<html><body style="font-family: sans-serif;">
                         <center>Timing Advance: {row.Timing_Advance}meters<br> 
                         <center>Users: {row.LTE_Users}<br> 
                         </body></html>""")
    pol.outerboundaryis = row.wedge_geometry.exterior.coords
    pol.style.linestyle.color = simplekml.Color.changealphaint(130, row.polycolor)
    pol.style.linestyle.width = 1
    pol.style.polystyle.color = simplekml.Color.changealphaint(130, row.polycolor)
kml.save("networkcellsummary.kml")